COMBINING MATCHES AND DELIVERIES TO CALCULATE ODDS PREDICTOR

In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
deliveries = pd.read_csv("../Data/deliveries_cleaned.csv")
matches = pd.read_csv("../Data/matches_cleaned.csv")

In [16]:
deliveries["match_no"] = deliveries["match_no"].apply(lambda x: float(x))

In [17]:
df = pd.merge(deliveries, matches, how = "inner", on = "match_no")

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 225792 entries, 0 to 225791
Data columns (total 22 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   batsman         225792 non-null  object 
 1   bowler          225792 non-null  object 
 2   over            225792 non-null  int64  
 3   ball            225792 non-null  int64  
 4   total_runs      225792 non-null  int64  
 5   batsman_runs    225792 non-null  int64  
 6   extras          225792 non-null  int64  
 7   match_no        225792 non-null  float64
 8   is_wicket       225792 non-null  float64
 9   dismissal_type  11122 non-null   object 
 10  fielder         8036 non-null    object 
 11  innings         225792 non-null  int64  
 12  date            225792 non-null  object 
 13  city            225792 non-null  object 
 14  team1           225792 non-null  object 
 15  team2           225792 non-null  object 
 16  toss            225792 non-null  object 
 17  decision  